In [4]:
import networkx as nx
import math
from collections import deque
import random
from itertools import combinations

def euclidean_distance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def obtezen_graf(n):
    while True:
        G = nx.Graph()
        
        # Add nodes with random positions
        for i in range(n):
            x = random.random()
            y = random.random()
            G.add_node(i, pos=(x, y))
        
        # Add edges between nodes within radius
        for i in range(n - 1):
            v1 = random.choice(list(G.nodes))
            v2 = random.choice(list(G.nodes - set([v1])))
            G.add_edge(v1, v2)
        if nx.is_connected(G):
            pos = nx.get_node_attributes(G, 'pos')
            # Assign weights to edges based on Euclidean distances
            for (u, v) in G.edges():
                G.edges[u, v]['weight'] = round(euclidean_distance(pos[u], pos[v]), 3)
            
            return G, pos

def bfs_shortest_paths(graph, start):
    shortest_paths = {}
    distances = {vertex: float('inf') for vertex in graph.nodes()}
    queue = deque([start])
    distances[start] = 0

    while queue:
        vertex = queue.popleft()
        for neighbor in graph.neighbors(vertex):
            edge_weight = graph.edges[vertex, neighbor]['weight']
            if distances[neighbor] > distances[vertex] + edge_weight:
                distances[neighbor] = distances[vertex] + edge_weight
                shortest_paths[neighbor] = shortest_paths.get(vertex, []) + [neighbor]
                queue.append(neighbor)

    return shortest_paths, distances

def vsota_razdalj(graf, v):
    shortest_paths, distances = bfs_shortest_paths(graf, v)
    vsota = 0
    for vertex, path in shortest_paths.items():
        vsota += distances[vertex]
    return vsota

def find_best_combination_for_node(graf, v, pos):
    current_edges = list(graf.edges(v, data=True))
    all_possible_edges_v = [(v, i) for i in graf.nodes if i != v]
    k = graf.degree(v)
    best_graph = graf.copy()
    best_distance_sum = vsota_razdalj(graf, v)

    for new_edges in combinations(all_possible_edges_v, k):
        test_graph = graf.copy()
        test_graph.remove_edges_from([(v, u) for v, u, _ in current_edges])
        for edge in new_edges:
            test_graph.add_edge(*edge)
            test_graph.edges[edge]['weight'] = round(euclidean_distance(pos[edge[0]], pos[edge[1]]), 3)
        
        if nx.is_connected(test_graph):
            new_distance_sum = vsota_razdalj(test_graph, v)
            if new_distance_sum < best_distance_sum:
                best_distance_sum = new_distance_sum
                best_graph = test_graph.copy()

    return best_graph


# Function to find the indexes of the highest values
def find_max_indexes(tuples_list):
    max_value = max(tuples_list, key=lambda x: x[1])[1]
    max_indexes = [index for index, value in tuples_list if value == max_value]
    return max_indexes

def find_min_indexes(tuples_list):
    max_value = min(tuples_list, key=lambda x: x[1])[1]
    max_indexes = [index for index, value in tuples_list if value == max_value]
    return max_indexes


def ekz_sum(graf, pos, visualize=False):
    def loop_sum(graf):
        ravnovesni_graf = graf.copy()
        for v in graf.nodes:
            testni_graf = find_best_combination_for_node(ravnovesni_graf, v, pos)
            if nx.is_connected(testni_graf) and vsota_razdalj(graf, v) > vsota_razdalj(testni_graf, v):
                        ravnovesni_graf = testni_graf.copy()
                        return loop_sum(ravnovesni_graf)
        return ravnovesni_graf
    
    return loop_sum(graf)


# Function to calculate the sum of direct distances from a node to all other nodes
def sum_direct_distances(graph, node, pos):
    total_distance = 0
    for other_node in graph.nodes:
        if other_node != node:
            total_distance += euclidean_distance(pos[node], pos[other_node])
    return total_distance

def n_m_c(g, poz):
    a_3 = [(n, sum_direct_distances(g, n, poz)) for n in g.nodes]
    return a_3


def vsota_r(graf, vozlisce):
    vsota = 0
    for v in graf.nodes - {vozlisce}:
        vsota += nx.shortest_path_length(graf, vozlisce, v)
    return vsota


def find_best(graf, v):
    current_edges = list(graf.edges(v, data=True))
    all_possible_edges_v = [(v, i) for i in graf.nodes if i != v]
    k = graf.degree(v)
    best_graph = graf.copy()
    best_distance_sum = vsota_r(graf, v)

    for new_edges in combinations(all_possible_edges_v, k):
        test_graph = graf.copy()
        test_graph.remove_edges_from([(v, u) for v, u, _ in current_edges])
        for edge in new_edges:
            test_graph.add_edge(*edge)
        
        if nx.is_connected(test_graph):
            new_distance_sum = vsota_r(test_graph, v)
            if new_distance_sum < best_distance_sum:
                best_distance_sum = new_distance_sum
                best_graph = test_graph.copy()

    return best_graph



def e_sum(graf):
    def loop_sum(graf):
        ravnovesni_graf = graf.copy()
        for v in graf.nodes:
            testni_graf = find_best(ravnovesni_graf, v)
            if nx.is_connected(testni_graf) and vsota_r(graf, v) > vsota_r(testni_graf, v):
                        ravnovesni_graf = testni_graf.copy()
                        return loop_sum(ravnovesni_graf)
        return ravnovesni_graf

    return loop_sum(graf)


def nakljucni_zacetni_graf(n):
    # Create an initial random graph with the specified number of nodes
    G = nx.gnm_random_graph(n, n - 1)
    # Ensure the graph is connected
    while not nx.is_connected(G):
        G = nx.gnm_random_graph(n, n - 1)
    return G


In [5]:
t = 100000
n = 4


vsota = [0] * n
for i in range(n):
    vsota[i] = 0

v_b_1 = [0] * n
for i in range(n):
    v_b_1[i] = 0

v_a_1 = [0] * n
for i in range(n):
    v_a_1[i] = 0

v_a_2 = [0] * n
for i in range(n):
    v_a_2[i] = 0

v_a_3 = [0] * n
for i in range(n):
    v_a_3[i] = 0

for i in range(t):
    g, poz = obtezen_graf(n)
    optimiziran = ekz_sum(g, poz, visualize=True)
    a_1 = g.degree()
    a_2 = [(v, vsota_razdalj(g, v)) for v in g.nodes]
    a_3 = n_m_c(g, poz)
    b_1 = optimiziran.degree()
    b_2 = [(v, vsota_razdalj(optimiziran, v)) for v in optimiziran.nodes]
    m_a_1 = max(a_1, key=lambda x: x[1])[1]
    m_a_2 = min(a_2, key=lambda x: x[1])[1]
    m_a_3 = min(a_3, key=lambda x: x[1])[1]
    m_b_1 = max(b_1, key=lambda x: x[1])[1]
    m_b_2 = min(b_2, key=lambda x: x[1])[1]
    mi_a_1 = find_max_indexes(a_1) 
    mi_a_2 = find_min_indexes(a_2) 
    mi_a_3 = find_min_indexes(a_3) 
    mi_b_1 = find_max_indexes(b_1)
    mi_b_2 = find_min_indexes(b_2)
    for j in mi_b_2:
        vsota[j] += 1
######################
    for j in mi_b_1:
        if j in mi_b_2:
            v_b_1[j] += 1
    for j in mi_a_1:
        if j in mi_b_2:
            v_a_1[j] += 1
    for j in mi_a_2:
        if j in mi_b_2:
            v_a_2[j] += 1
    for j in mi_a_3:
        if j in mi_b_2:
            v_a_3[j] += 1

s_b_1 = sum(v_b_1)
s_a_1 = sum(v_a_1)
s_a_2 = sum(v_a_2)
s_a_3 = sum(v_a_3)

print(vsota, sum(vsota)) # indeks najcenejšega
print(v_b_1, s_b_1) # indeks največ povezav na koncu tudi zmaga

print(v_a_1, s_a_1) # indeks največ povezav na začetku zmaga
print(v_a_2, s_a_2) # indeks z začetno najmanjšo ceno zmaga
print(v_a_3, s_a_3) # indeks z najmanjšo možno ceno zmaga

[38430, 38550, 38682, 38889] 154551
[38430, 38550, 38682, 38889] 154551
[17164, 17086, 17102, 17180] 68532
[17164, 17086, 17102, 17180] 68532
[24069, 24034, 24195, 24272] 96570


In [6]:
vsota = [0] * n
for i in range(n):
    vsota[i] = 0

v_b_1 = [0] * n
for i in range(n):
    v_b_1[i] = 0

v_a_1 = [0] * n
for i in range(n):
    v_a_1[i] = 0

v_a_2 = [0] * n
for i in range(n):
    v_a_2[i] = 0


for i in range(t):
    g = nakljucni_zacetni_graf(n)
    optimiziran = e_sum(g)
    a_1 = g.degree()
    a_2 = [(v, vsota_r(g, v)) for v in g.nodes]
    b_1 = optimiziran.degree()
    b_2 = [(v, vsota_r(optimiziran, v)) for v in optimiziran.nodes]
    m_a_1 = max(a_1, key=lambda x: x[1])[1]
    m_a_2 = min(a_2, key=lambda x: x[1])[1]
    m_b_1 = max(b_1, key=lambda x: x[1])[1]
    m_b_2 = min(b_2, key=lambda x: x[1])[1]
    mi_a_1 = find_max_indexes(a_1)
    mi_a_2 = find_min_indexes(a_2)
    mi_b_1 = find_max_indexes(b_1)
    mi_b_2 = find_min_indexes(b_2)
    for j in mi_b_2:
        vsota[j] += 1
######################
    for j in mi_b_1:
        if j in mi_b_2:
            v_b_1[j] += 1
    for j in mi_a_1:
        if j in mi_b_2:
            v_a_1[j] += 1
    for j in mi_a_2:
        if j in mi_b_2:
            v_a_2[j] += 1

s_b_1 = sum(v_b_1)
s_a_1 = sum(v_a_1)
s_a_2 = sum(v_a_2)

print(vsota, sum(vsota)) # indeks najcenejšega
print(v_b_1, s_b_1) # indeks največ povezav na koncu tudi zmaga

print(v_a_1, s_a_1) # indeks največ povezav na začetku zmaga
print(v_a_2, s_a_2) # indeks z začetno najmanjšo ceno zmaga

[25066, 24866, 25031, 25037] 100000
[25066, 24866, 25031, 25037] 100000
[25066, 24866, 25031, 25037] 100000
[25066, 24866, 25031, 25037] 100000
